# Depeche Mode lyrics generator using a LSTM neural network

Based on the excellent example at https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [ ]:
import numpy as np
import warnings
import sys
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

warnings.filterwarnings('ignore')

In [ ]:
filename = 'dm_lyrics.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

# character-to-integer mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print('total characters: ', n_chars)
print('total vocabulary: ', n_vocab)

In [ ]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i+seq_length]
    seq_out = raw_text[i+seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)
print('total patterns: ', n_patterns)

In [ ]:
''' 
    sequence input format expected by the LSTM-based neural network:
    [samples, timesteps, features]
'''
# reshape X to the expected input format
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalization
X = X / float(n_vocab)

# one-hot encoding of the output
y = to_categorical(dataY)

print(X.shape, y.shape)

In [123]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 100, 256)          264192    
_________________________________________________________________
dropout_9 (Dropout)          (None, 100, 256)          0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dropout_10 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 44)                11308     
Total params: 800,812
Trainable params: 800,812
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# define the checkpoint
filepath = 'weights_improvement_{epoch:02d}_{loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(X, y, epochs=3, batch_size=128, callbacks=[checkpoint])

## notebook client using the trained model

In [158]:
# load the network weights
filename = 'weights_improvement_10_0.9644.h5'

model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

int_to_char = dict((i, c) for i, c in enumerate(chars))

In [166]:
''' make predictions '''
# pick a random seed value to start
# with a series of real lyrics
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print('seed: ')
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
print('\n\n\n****[START SYNTHETIC LYRICS]****\n\n\n')
for i in range(250):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

print('\n\n\n****[FIN]****')

seed: 
" a time  
when all on my mind was love  
now i find that most of the time  
love's not enough in itse "



****[START SYNTHETIC LYRICS]****



lf  
that she sain and the still lead  
the still in the sky  
ray you're sucying  
to make you  
sometimes i don't know  
the wrong reason and the srill is ceaua  
the sains that i want to be with me  
i'm almande that thou hor to searct and searet 


****[FIN]****
